In [1]:
# profiling.py
"""Python Essentials: Profiling.
Nathan Schill
Section 2
Tues. Mar. 28, 2023
"""

# Note: for problems 1-4, you need only implement the second function listed.
# For example, you need to write max_path_fast(), but keep max_path() unchanged
# so you can do a before-and-after comparison.

import numpy as np
# from numba import jit

In [2]:
# Problem 1
def max_path(filename='triangle.txt'):
    """Find the maximum vertical path in a triangle of values."""
    with open(filename, 'r') as infile:
        data = [[int(n) for n in line.split()]
                        for line in infile.readlines()]
    def path_sum(r, c, total):
        """Recursively compute the max sum of the path starting in row r
        and column c, given the current total.
        """
        total += data[r][c]
        if r == len(data) - 1:          # Base case.
            return total
        else:                           # Recursive case.
            return max(path_sum(r+1, c,   total),   # Next row, same column
                       path_sum(r+1, c+1, total))   # Next row, next column

    return path_sum(0, 0, 0)            # Start the recursion from the top.

def max_path_fast(filename='triangle_large.txt'):
    """Find the maximum vertical path in a triangle of values."""

    # Read file into list of lists
    with open(filename, 'r') as infile:
        data = [[int(n) for n in line.split()]
                        for line in infile.readlines()]
    
    # Start with penultimate row
    for i in range(len(data)-2, -1, -1):
        # Iterate across entries
        for j in range(len(data[i])):
            # Add to the current entry the max of the two entries below
            data[i][j] += max(data[i+1][j], data[i+1][j+1])
    
    # Max path sum is at top of triangle
    return data[0][0]

In [ ]:
%timeit max_path()
%timeit max_path_fast('triangle.txt')

11 ms ± 687 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
102 µs ± 29.1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [23]:
# Problem 2
### Hint: Prime factorizations -> only need to check primes
def primes(N):
    """Compute the first N primes."""
    primes_list = []
    current = 2
    while len(primes_list) < N:
        isprime = True
        for i in range(2, current):     # Check for nontrivial divisors.
            if current % i == 0:
                isprime = False
        if isprime:
            primes_list.append(current)
        current += 1
    return primes_list

def primes_fast(N):
    """Compute the first N primes."""

    # Init with 2 since the only even prime
    primes = [2] + [None] * (N-1)
    num_primes = 1

    # Start at 3 and iterate until have N primes
    current = 3
    while num_primes < N:
        # Store the square root of current
        _sqrt = current**(1/2)
        isprime = True
        
        # Check whether each known prime (skipping 2) less than _sqrt divides current
        for p in primes[1:num_primes]:
            if current % p == 0:
                # Not prime
                isprime = False
                break
            
            # Don't need to check any more primes
            if p >= _sqrt:
                # No prime divides current, so current is prime
                break
        
        if isprime:
            primes[num_primes] = current
            num_primes += 1
        current += 2
    
    return primes


In [ ]:
### Using a set to store primes instead of a list
def primes_fast(N):
    """Compute the first N primes."""

    # Init with 2 since the only even prime
    primes = {2}
    num_primes = 1

    # Start at 3 and iterate until have N primes
    current = 3
    while num_primes < N:
        # Store the square root of current
        _sqrt = current**(1/2)
        
        # Check whether each known prime less than _sqrt divides current
        for p in primes:
            
            if current % p == 0:
                # Not prime
                break
            
            ### TODO: Sets don't iterate in insertion order
            # Don't need to check any more primes
            if p >= _sqrt:
                # No prime divides current, so current is prime
                primes.add(current)
                num_primes += 1
                break
        
        current += 2
    
    return primes

In [24]:
N = 10
primes(N), primes_fast(N)

([2, 3, 5, 7, 11, 13, 17, 19, 23, 29], [2, 3, 5, 7, 11, 13, 17, 19, 23, 29])

In [25]:
### TODO
%time primes_fast(10000)
print()

CPU times: total: 1.27 s
Wall time: 1.35 s



In [ ]:
# Problem 3
def nearest_column(A, x):
    """Find the index of the column of A that is closest to x.

    Parameters:
        A ((m,n) ndarray)
        x ((m, ) ndarray)

    Returns:
        (int): The index of the column of A that is closest in norm to x.
    """
    distances = []
    for j in range(A.shape[1]):
        distances.append(np.linalg.norm(A[:,j] - x))
    
    return np.argmin(distances)

def nearest_column_fast(A, x):
    """Find the index of the column of A that is closest in norm to x.
    Refrain from using any loops or list comprehensions.

    Parameters:
        A ((m,n) ndarray)
        x ((m, ) ndarray)

    Returns:
        (int): The index of the column of A that is closest in norm to x.
    """
    
    return np.argmin(np.linalg.norm(A-x[:, np.newaxis], axis=0))

A = np.array([[1, 2, 3], [1, 2, 3]])
x = np.array([1, 1])

nearest_column_fast(A, x)

0

In [ ]:
N = 100
A = np.random.random((N, N))
x = np.random.random(N)

%time nearest_column(A, x)
%time nearest_column_fast(A, x)

CPU times: user 2.47 ms, sys: 31 µs, total: 2.5 ms
Wall time: 1.82 ms
CPU times: user 147 µs, sys: 0 ns, total: 147 µs
Wall time: 150 µs


58

In [ ]:
# Problem 4
### Hint: Use comprehensions (list, dictionary, etc.) and NumPy array operations
def name_scores(filename="names.txt"):
    """Find the total of the name scores in the given file."""
    with open(filename, 'r') as infile:
        names = sorted(infile.read().replace('"', '').split(','))
    total = 0
    for i in range(len(names)):
        name_value = 0
        for j in range(len(names[i])):
            alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
            for k in range(len(alphabet)):
                if names[i][j] == alphabet[k]:
                    letter_value = k + 1
            name_value += letter_value
        total += (names.index(names[i]) + 1) * name_value
    return total

def name_scores_fast(filename='names.txt'):
    """Find the total of the name scores in the given file."""
    raise NotImplementedError("Problem 4 Incomplete")




In [ ]:
# Problem 5
def fibonacci():
    """Yield the terms of the Fibonacci sequence with F_1 = F_2 = 1."""
    raise NotImplementedError("Problem 5 Incomplete")

def fibonacci_digits(N=1000):
    """Return the index of the first term in the Fibonacci sequence with
    N digits.

    Returns:
        (int): The index.
    """
    raise NotImplementedError("Problem 5 Incomplete")




In [ ]:
# Problem 6
### Numba: lab machines, ssh lab machines, or Colab
def prime_sieve(N):
    """Yield all primes that are less than N."""
    raise NotImplementedError("Problem 6 Incomplete")




In [ ]:
# Problem 7
def matrix_power(A, n):
    """Compute A^n, the n-th power of the matrix A."""
    product = A.copy()
    temporary_array = np.empty_like(A[0])
    m = A.shape[0]
    for power in range(1, n):
        for i in range(m):
            for j in range(m):
                total = 0
                for k in range(m):
                    total += product[i,k] * A[k,j]
                temporary_array[j] = total
            product[i] = temporary_array
    return product

def matrix_power_numba(A, n):
    """Compute A^n, the n-th power of the matrix A, with Numba optimization."""
    raise NotImplementedError("Problem 7 Incomplete")



In [ ]:
def prob7(n=10):
    """Time matrix_power(), matrix_power_numba(), and np.linalg.matrix_power()
    on square matrices of increasing size. Plot the times versus the size.
    """
    raise NotImplementedError("Problem 7 Incomplete")
